In [40]:
from pyspark.sql import SparkSession

In [41]:
spark = SparkSession.builder.appName("CsvToKafka").config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0") \
    .getOrCreate()
print(spark)

In [42]:
df = spark.read.csv("file:///home/hdoop/notebooks/data/spark_practice/walmart_stock.csv",header=True,inferSchema=True)

In [43]:
df.printSchema()

root
 |-- Date: date (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



In [44]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

In [45]:
schema = StructType([
    StructField("customer_id",IntegerType(),True),
    StructField("transaction_type",StringType(),True),
    StructField("transaction_amount",IntegerType(),True)
])


In [46]:

df = spark.readStream.format("csv").option("Header","True").schema(schema).load("file:///home/hdoop/notebooks/data/spark_practice/csvToKafka/*")

In [47]:
# query = df.writeStream.format("console").outputMode("append").start().awaitTermination()

In [48]:
df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- transaction_type: string (nullable = true)
 |-- transaction_amount: integer (nullable = true)



In [49]:
# df.writeStream.format("console").outputMode("update").start()

In [50]:
# query = df2.writeStream.format("kafka")\
# .option("kafka.bootstrap.server","localhost:9092")\
# .option("topic","listen")\
# .start()\
# .awaitTermination()

In [51]:
df2 = df.selectExpr("CAST(customer_id AS STRING) as key", " to_json(struct(*)) AS value")


In [52]:
query = df2.writeStream.format("kafka")\
    .option("kafka.bootstrap.servers", "localhost:9092")\
    .option("topic", "listen")\
    .option("checkpointLocation", "file:///home/hdoop/notebooks/checkpoint/csvToKafka/")\
    .start()\
    .awaitTermination()


25/03/31 15:24:01 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/03/31 15:24:01 WARN StreamingQueryManager: Stopping existing streaming query [id=ebf3d091-72f3-482f-86e2-a1ea28894a25, runId=2e7edb32-ec07-4229-b575-c0829d1b1cb3], as a new run is being started.
25/03/31 15:24:01 WARN HadoopFSUtils: The directory file:/home/hdoop/notebooks/data/spark_practice/csvToKafka/untitled.txt was not found. Was it deleted very recently?
ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/home/hdoop/.local/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/hdoop/.local/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_

KeyboardInterrupt: 

In [53]:
spark.stop()